In [1]:
import pandas as pd

# Lectura de datos

In [2]:
df_caracteristicas_equipos = pd.read_csv('../data/Caracteristicas_Equipos.csv')
df_historicos_ordenes = pd.read_csv('../data/Historicos_Ordenes.csv')
df_registros_condiciones = pd.read_csv('../data/Registros_Condiciones.csv')

# 1. Preparación DF

In [3]:
def convertir_fecha_a_datetime(df, columna):
    df[columna] = pd.to_datetime(df[columna])
    return df

def cantidad_nulos_por_cada_columna(df, nombre_df):
    print ("--------------------------------------------------")
    print (f'Cantidad de nulos por cada columna de {nombre_df}')
    print (df.isnull().sum().to_frame('Total_Nulos').T)

def mostrar_duplicados_por_columna(df, nombre_columna, nombre_df):
    print ("--------------------------------------------------")
    duplicados = df[df.duplicated(subset=[nombre_columna])]
    print (f'Duplicados en la columna {nombre_columna} de {nombre_df}')
    print (duplicados)
    print (f'Cantidad de duplicados: {len(duplicados)}')
    # Eliminar duplicados y mantener la primera ocurrencia
    df_sin_duplicados = df.drop_duplicates(subset=[nombre_columna], keep='first')
    print(f'\nSe eliminaron {len(df) - len(df_sin_duplicados)} registros duplicados')
    return df_sin_duplicados

### Se convierten a fechas los campos Fecha de cada DF

In [4]:
df_historicos_ordenes = convertir_fecha_a_datetime(df_historicos_ordenes, 'Fecha')
df_registros_condiciones = convertir_fecha_a_datetime(df_registros_condiciones, 'Fecha')

### Se comprueban los nulos
### Se observa que los nulos afectarán al modelo. !No se eliminan!!

In [5]:
cantidad_nulos_por_cada_columna(df_caracteristicas_equipos, 'df_caracteristicas_equipos')
print ()
cantidad_nulos_por_cada_columna(df_historicos_ordenes, 'df_historicos_ordenes')
print()
cantidad_nulos_por_cada_columna(df_registros_condiciones, 'df_registros_condiciones')

--------------------------------------------------
Cantidad de nulos por cada columna de df_caracteristicas_equipos
             ID_Equipo  Tipo_Equipo  Fabricante  Modelo  Potencia_kW  \
Total_Nulos          0            0           0       0            0   

             Horas_Recomendadas_Revision  
Total_Nulos                            0  

--------------------------------------------------
Cantidad de nulos por cada columna de df_historicos_ordenes
             ID_Orden  ID_Equipo  Fecha  Tipo_Mantenimiento  \
Total_Nulos         0          0      0                   0   

             Costo_Mantenimiento  Duracion_Horas  Ubicacion  
Total_Nulos                   49               0          0  

--------------------------------------------------
Cantidad de nulos por cada columna de df_registros_condiciones
             ID_Registro  ID_Equipo  Fecha  Temperatura_C  Vibracion_mm_s  \
Total_Nulos            0          0      0              0               0   

             Horas_O

### Se muestran los duplicados y se eliminan

In [6]:
mostrar_duplicados_por_columna(df_caracteristicas_equipos, 'ID_Equipo', 'df_caracteristicas_equipos')
mostrar_duplicados_por_columna(df_historicos_ordenes, 'ID_Orden', 'df_historicos_ordenes')
mostrar_duplicados_por_columna(df_registros_condiciones, 'ID_Registro', 'df_registros_condiciones')

--------------------------------------------------
Duplicados en la columna ID_Equipo de df_caracteristicas_equipos
     ID_Equipo    Tipo_Equipo Fabricante Modelo  Potencia_kW  \
500          1      Compresor    Siemens   Z300       3429.0   
501          2  Transformador    Siemens   Y200         75.0   
502          3      Compresor        ABB   Z300       4526.0   
503          4      Compresor         GE   X100       3981.0   
504          5          Motor    Siemens   Y200        377.0   

     Horas_Recomendadas_Revision  
500                         7725  
501                         7390  
502                         5238  
503                         8933  
504                         5978  
Cantidad de duplicados: 5

Se eliminaron 5 registros duplicados
--------------------------------------------------
Duplicados en la columna ID_Orden de df_historicos_ordenes
       ID_Orden  ID_Equipo               Fecha Tipo_Mantenimiento  \
10000         1         36 2020-01-01 00:00:00

,ID_Registro,ID_Equipo,Fecha,Temperatura_C,Vibracion_mm_s,Horas_Operativas
0,1,260,2020-01-01 00:00:00,101.43,1.22,71849.0
1,2,376,2020-01-01 01:00:00,118.51,2.97,49670.0
2,3,443,2020-01-01 02:00:00,76.04,2.93,33106.0
3,4,281,2020-01-01 03:00:00,97.09,3.25,31744.0
4,5,427,2020-01-01 04:00:00,139.42,4.22,78104.0
...,...,...,...,...,...,...
8995,8996,299,2021-01-09 19:00:00,51.08,2.32,97743.0
8996,8997,160,2021-01-09 20:00:00,122.18,2.65,84374.0
8997,8998,160,2021-01-09 21:00:00,119.70,2.98,16939.0
8998,8999,45,2021-01-09 22:00:00,57.77,3.39,86679.0


### 1. Explorar usando YDATA PROFILING


In [7]:
from ydata_profiling import ProfileReport


profile = ProfileReport(df_caracteristicas_equipos, title="Caracteristicas Equipos Profiling Report",
                        correlations={
                            "pearson": {"calculate": True},
                            "cramers": {"calculate": True},
                        })
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 107.15it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Creación features

### Añadimos a cada orden el número de horas recomendas para el ID_Equipo

In [8]:
# Hacemos el merge usando 'ID_Equipo' y eliminamos duplicados
df_trabajo_modelo = df_historicos_ordenes.merge(
    df_caracteristicas_equipos[['ID_Equipo', 'Horas_Recomendadas_Revision']],
    on='ID_Equipo',
    how='left'  # Utilizamos left join para mantener todos los registros del primer DataFrame
).drop_duplicates(subset=['ID_Equipo', 'Fecha'])

### Añadirmos el tiempo transcurrido en horas desde el último mantenimiento

In [9]:
# Ordenamos el DataFrame por ID_Equipo y Fecha
df_trabajo_modelo = df_trabajo_modelo.sort_values(['ID_Equipo', 'Fecha'])

# Calculamos la diferencia de tiempo con la orden anterior para cada equipo
df_trabajo_modelo['Horas_Desde_Ultima_Orden'] = df_trabajo_modelo.groupby('ID_Equipo')['Fecha'].diff().dt.total_seconds() / 3600

# Reemplazamos los NaN (que corresponden a la primera orden de cada equipo) con 0
df_trabajo_modelo['Horas_Desde_Ultima_Orden'] = df_trabajo_modelo['Horas_Desde_Ultima_Orden'].fillna(0)

### Añadimos las condiciones en el momento del mantenimiento usando las medidas más próximas

In [10]:

# Preparamos el DataFrame de registros de condiciones seleccionando solo las columnas necesarias
df_condiciones_reducido = df_registros_condiciones[['Fecha', 'ID_Equipo', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']]

# Realizamos un merge asíncrono para encontrar los registros más cercanos en tiempo
df_trabajo = pd.merge_asof(
    df_trabajo_modelo.sort_values('Fecha'),
    df_condiciones_reducido.sort_values('Fecha'),
    by='ID_Equipo',
    on='Fecha',
    direction='nearest'
)
df_trabajo_modelo = df_trabajo
print (df_trabajo_modelo.head())


   ID_Orden  ID_Equipo               Fecha Tipo_Mantenimiento  \
0         1         36 2020-01-01 00:00:00         Preventivo   
1         2        244 2020-01-01 01:00:00         Preventivo   
2         3        297 2020-01-01 02:00:00         Preventivo   
3         4        431 2020-01-01 03:00:00         Correctivo   
4         5        403 2020-01-01 04:00:00         Correctivo   

   Costo_Mantenimiento  Duracion_Horas     Ubicacion  \
0              6754.12               3  Planta Norte   
1              4274.63              35  Planta Norte   
2              5752.00              25  Planta Norte   
3              5690.14               8    Planta Sur   
4              7048.79              28  Planta Norte   

   Horas_Recomendadas_Revision  Horas_Desde_Ultima_Orden  Temperatura_C  \
0                         3549                       0.0         139.97   
1                         1061                       0.0         141.25   
2                         8016                 

## Candidatos

1. `LogisticRegression` # No coincide con perfil de datos, para comparar
2. `RandomForestClassifier` 
3. `GradientBoostingClassifier`

In [11]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer


In [12]:
target_column = "Tipo_Mantenimiento"

# Separar características y target
X = df_trabajo_modelo.drop(target_column, axis=1)
X = X.drop(columns=['Fecha'])
X = X.drop(columns=['Ubicacion'])
X = X.drop(columns=['ID_Orden'])


#y = pd.get_dummies(df_trabajo_modelo[target_column], drop_first=True)
y = pd.get_dummies(df_trabajo_modelo[target_column], drop_first=True).values.reshape(-1, 1).ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



### Pipelines

In [13]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)

print("Numerical columns", numerical_columns)
print("Categorical columns", categorical_columns)

Numerical columns ['ID_Equipo', 'Costo_Mantenimiento', 'Duracion_Horas', 'Horas_Recomendadas_Revision', 'Horas_Desde_Ultima_Orden', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas']
Categorical columns []


In [14]:

preprocessor = ColumnTransformer(
    [
        ('numerical', Pipeline([
            ('imputer', SimpleImputer(strategy='mean')),
            ('scaler', StandardScaler())
        ]), numerical_columns),
        ('categorical', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_columns)
    ])

pipelines = {
    "LogisticRegression": Pipeline([('preprocessor', preprocessor),('classifier', LogisticRegression())]),
    "RandomForestClassifier": Pipeline([('preprocessor', preprocessor),('classifier', RandomForestClassifier())]),
    "GradientBoostingClassifier": Pipeline([('preprocessor', preprocessor),('classifier', GradientBoostingClassifier())]),
}

In [15]:
def cv_train(name, pipeline, cv):
    print(f"Entrenar {name}")
    cv_results = cross_validate(pipeline, X_train, y_train, cv=cv, scoring="accuracy", return_estimator=True, return_train_score=True)
    trained_model = cv_results["estimator"][0]
    scores = pd.DataFrame(cv_results)

    print("test score (mean-std): {0:.2f} - {1:.2f}".format(scores["test_score"].mean(), scores["test_score"].std()))
    print("train score (mean-std): {0:.2f} - {1:.2f}".format(scores["train_score"].mean(), scores["train_score"].std()))
    print("params:", pipeline.named_steps.get("classifier").get_params())

    y_pred = trained_model.predict(X_test)
    f1 = f1_score(y_test, y_pred)

    return {"acc": round(scores["test_score"].mean(), 2), "f1": round(f1, 2),}

In [ ]:
cvss = ShuffleSplit(n_splits=40, test_size=0.2, random_state=0)

results = {}

for name, pipeline in pipelines.items():
    results[name] = cv_train(name, pipeline, cvss)

print("Resultados:")
results_df=pd.DataFrame(results)
results_df

Entrenar LogisticRegression
test score (mean-std): 0.49 - 0.01
train score (mean-std): 0.51 - 0.00
params: {'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
Entrenar RandomForestClassifier
